## Import statements

In [1]:
import json
import wget
import branca
import folium
import requests
import pandas as pd
from folium import plugins
from datetime import datetime
from bs4 import BeautifulSoup

## Scrape the data

In [2]:
BASE_URL = 'https://www.health.govt.nz/'
TODAY = datetime.today().strftime('%Y-%m-%d')

CURRENT_CASE_PAGE = BASE_URL + 'our-work/diseases-and-conditions/covid-19-novel-coronavirus/covid-19-current-situation/covid-19-current-cases'

r = requests.get(CURRENT_CASE_PAGE)
soup = BeautifulSoup(r.text, 'html.parser')
tables = soup.select('table.table-style-two')
cases = tables[1].select('tr')[1:-1]

f = open(f'data/{TODAY}.csv', 'w')
f.write('dhb,cases\n')
for case in cases:
    data = case.select('td')
    f.write(f'{data[0].get_text()},{data[1].get_text()}\n')
f.close()

CASE_DETAIL_PAGE = BASE_URL + 'our-work/diseases-and-conditions/covid-19-novel-coronavirus/covid-19-current-situation/covid-19-current-cases/covid-19-current-cases-details'

r = requests.get(CASE_DETAIL_PAGE)
soup = BeautifulSoup(r.text, 'html.parser')
excel_download_link = soup.find(lambda tag:tag.name=="a" and "Confirmed COVID-19 cases" in tag.text)['href']
wget.download(BASE_URL + excel_download_link, f'data/case_details/{TODAY}.xlsx')

'data/case_details/2020-03-28 (1).xlsx'

## Loading the data

In [3]:
dhb_geo = json.load(open('dhb.geojson'))
df = pd.read_csv(f'data/{TODAY}.csv')
case_series = df.set_index('dhb')['cases']

# combine the data
for feature in dhb_geo['features']:
    feature['properties']['cases'] = int(case_series[feature['properties']['DHB2015_Na']])

## Initialize the map

In [4]:
m = folium.Map(
    location=['-41.280710', '175.070684'],
    tiles='CartoDB positron',
    zoom_start=5,
    zoom_control=False,
)

# Colorize the region using a color scale
colorscale = branca.colormap.linear.YlGnBu_09.scale(0, case_series.quantile(1))
colorscale.caption="Confirmed COVID-19 cases in NZ by District Health Board"
colorscale.add_to(m)

def style_function(feature):
    case = case_series.get(feature['properties']['DHB2015_Na'])
    return {
        'fillOpacity': 0.5,
        'fillColor': colorscale(case),
        'weight': 0.5,
        'color': 'black',
    }

# Add tooltip on hover
tooltip = folium.features.GeoJsonTooltip(
    fields=["DHB2015_Na", "cases"],
    aliases=["DHB: ", "# of cases: "],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
)

# Add DHB geojson to map
folium.GeoJson(
    dhb_geo,
    name='geojson',
    style_function=style_function,
    tooltip=tooltip,
).add_to(m)

plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True,
).add_to(m)

m.save('index.html')
m